In [1]:
from glob import glob

In [2]:
imgs = glob("static/imgs/*.jpg")

In [53]:
import numpy as np

def next_vig(available, i, max_vigilence):
    l = []
    for j in range(i+1, len(available)):
        if available[j] - available[i] <= max_vigilence:
            l.append(j)
        else:
            break
    if len(l) == 0:
        return None
    return np.random.choice(l, 1)[0]

def get_sequence(n=178, n_targets=66, target_gap=80, n_filler=22, n_vigilence=12, max_vigilence=10):
    label_list = ["filler", "target", "target_rep", "vigilence", "vigilence_rep"]
    labels = np.zeros(n)
    targets = np.random.choice(n-target_gap, n_targets, replace=False)
    labels[targets] = 1
    labels[targets+target_gap] = 2
    available = np.array(range(n))[labels == 0]

    current_vigilence = 0
    for i in range(n_vigilence*2):
        s = np.random.choice(len(available), 1)[0]
        s_next = next_vig(available, s, max_vigilence)
        if s_next is None:
            continue
        labels[s] = 3
        labels[s_next] = 4
        np.delete(available, s)
        np.delete(available, s_next)
        current_vigilence += 1
        if current_vigilence == n_vigilence:
            break
    return labels

In [47]:
n=178; n_targets=66; target_gap=80; n_filler=22; n_vigilence=12; max_vigilence=10

labels = np.zeros(n)
targets = np.random.choice(n-target_gap, n_targets, replace=False)

labels[targets] = 1
labels[targets+target_gap] = 2

In [54]:
get_sequence(n=178, n_targets=66, target_gap=80, n_filler=22, n_vigilence=12, max_vigilence=10)

array([3., 1., 1., 4., 3., 3., 4., 1., 3., 1., 0., 4., 0., 0., 0., 1., 0.,
       1., 1., 3., 4., 3., 4., 3., 3., 4., 1., 0., 1., 0., 1., 3., 4., 1.,
       1., 1., 0., 0., 1., 0., 1., 3., 1., 0., 4., 0., 0., 0., 1., 1., 1.,
       1., 3., 4., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 1., 2., 0., 0., 1., 2., 1., 2., 2., 2., 0., 2.,
       2., 2., 2., 0., 2., 0., 2., 0., 2., 2., 2., 2., 2., 2., 0., 0., 2.,
       0., 2., 0., 2., 0., 2., 0., 0., 0., 2., 2., 2., 2., 0., 2., 2., 0.,
       0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 0., 2., 2.,
       2., 0., 2., 2., 2., 0., 2., 0., 2., 2., 0., 2., 0., 2., 2., 2., 2.,
       2., 2., 0., 0., 2., 0., 2., 2.])